# **From Planning to Execution: Building Intelligent Agents with LangGraph**

## Introduction

The provided example demonstrates a **multi-step task execution workflow** using LangChain and LangGraph. The workflow is designed to handle complex queries by breaking them down into smaller, manageable steps, executing them sequentially, and dynamically re-planning as needed. This approach ensures that the agent can handle tasks that require multiple steps, such as searching for information, processing results, and providing a final response.

### Key Steps in the Workflow:
1. **Initialize the LLM**: The workflow begins by initializing a language model (LLM) using either OpenAI's GPT-4 or Anthropic's Claude-3. The API key is securely fetched using `kaggle_secrets`.
2. **Define Tools and Custom Prompt**: The workflow defines the tools (e.g., `DuckDuckGoSearchRun`) and a custom prompt to guide the agent's behavior.
3. **Define the State and Planning Logic**: The state structure (`PlanExecute`) is defined to track the input, plan, past steps, and final response. The `planner` generates a step-by-step plan to achieve the objective.
4. **Define Re-Planning Logic**: The `replanner` updates the plan based on the results of previous steps. If no further steps are needed, it returns the final response.
5. **Create the Graph and Define Nodes**: The workflow graph is created, with nodes for planning (`plan_node`), execution (`agent_node`), and re-planning (`replan_node`). The graph orchestrates the execution of tasks and ensures the workflow terminates when the final response is ready.
6. **Execute the Workflow**: The workflow is executed with a sample input, and the agent processes the query step by step, providing the final result.

---

## Preparation

### Installing Required Libraries
This section installs the necessary Python libraries for working with LangChain, OpenAI embeddings, Anthropic models, DuckDuckGo search, and other utilities. These libraries include:
- `langchain-openai`: Provides integration with OpenAI's embedding models and APIs.
- `langchain-anthropic`: Enables integration with Anthropic's models and APIs.
- `langchain_community`: Contains community-contributed modules and tools for LangChain.
- `langchain_experimental`: Includes experimental features and utilities for LangChain.
- `langgraph`: A library for building and visualizing graph-based workflows in LangChain.
- `duckduckgo-search`: Enables programmatic access to DuckDuckGo's search functionality.

In [ ]:
!pip install -qU langchain-openai
!pip install -qU langchain-anthropic
!pip install -qU langchain_community
!pip install -qU langchain_experimental
!pip install -qU langgraph
!pip install -qU duckduckgo-search

---

## Step 1: Initialize LLM with API Key
This step initializes the language model (LLM) using either OpenAI's GPT-4 or Anthropic's Claude-3. The API key is securely fetched using `kaggle_secrets`.

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_anthropic import ChatAnthropic
from kaggle_secrets import UserSecretsClient

# Fetch API key securely
user_secrets = UserSecretsClient()

# Initialize LLM
model = ChatOpenAI(model="gpt-4o", temperature=0, api_key=user_secrets.get_secret("my-openai-api-key"))
#model = ChatAnthropic(model="claude-3-5-sonnet-latest", temperature=0, api_key=user_secrets.get_secret("my-anthropic-api-key"))

## Step 2: Define Tools and Custom Prompt
This step defines the tools (e.g., `DuckDuckGoSearchRun`) and a custom prompt for the agent. The prompt instructs the agent to execute tasks step by step.

In [ ]:
import operator
from typing import Annotated, List, Tuple, Union, Literal
from typing_extensions import TypedDict
from pydantic import BaseModel, Field
from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.prompts import ChatPromptTemplate
from langgraph.graph import StateGraph, START, END
from langgraph.prebuilt import create_react_agent

# Define Tools
tools = [DuckDuckGoSearchRun()]

# Define Custom Prompt
custom_prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant that can execute tasks step by step.")
])

# Define Execution Agent
agent_executor = create_react_agent(model, tools, state_modifier=custom_prompt)

## Step 3: Define the State and Planning Logic
This step defines the state structure (`PlanExecute`) and the planning logic. The `Plan` class represents the steps to achieve the objective, and the `planner` generates a step-by-step plan.

In [ ]:
# Define the State
class PlanExecute(TypedDict):
    """
    Represents the state of the execution workflow.
    
    Attributes:
        input (str): The user's input or objective.
        plan (List[str]): A list of steps to achieve the objective.
        past_steps (List[Tuple]): A list of tuples representing completed steps and their results.
        response (str): The final response to the user.
    """
    input: str
    plan: List[str]
    past_steps: Annotated[List[Tuple], operator.add]
    response: str

# Planning Step
class Plan(BaseModel):
    """
    Represents a plan consisting of steps to achieve an objective.
    
    Attributes:
        steps (List[str]): A list of steps to follow, sorted in the required order.
    """
    steps: List[str] = Field(description="different steps to follow, should be in sorted order")

planner_prompt = ChatPromptTemplate.from_messages([
    ("system", """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps."""),
    ("placeholder", "{messages}")
])

planner = planner_prompt | model.with_structured_output(Plan)

## Step 4: Define Re-Planning Logic
This step defines the re-planning logic. The `replanner` updates the plan based on the results of previous steps. If no further steps are needed, it returns the final response.

In [ ]:
# Re-Plan Step
class Response(BaseModel):
    """
    Represents a response to the user.
    
    Attributes:
        response (str): The final response message.
    """
    response: str

class Act(BaseModel):
    """
    Represents an action to perform, which can be either a response or a new plan.
    
    Attributes:
        action (Union[Response, Plan]): The action to perform. Use `Response` to respond to the user or `Plan` to continue executing steps.
    """
    action: Union[Response, Plan] = Field(description="Action to perform. If you want to respond to user, use Response. If you need to further use tools to get the answer, use Plan.")

replanner_prompt = ChatPromptTemplate.from_template(
    """For the given objective, come up with a simple step by step plan. \
This plan should involve individual tasks, that if executed correctly will yield the correct answer. Do not add any superfluous steps. \
The result of the final step should be the final answer. Make sure that each step has all the information needed - do not skip steps.

Your objective was this:
{input}

Your original plan was this:
{plan}

You have currently done the follow steps:
{past_steps}

Update your plan accordingly. If no more steps are needed and you can return to the user, then respond with that. Otherwise, fill out the plan. Only add steps to the plan that still NEED to be done. Do not return previously done steps as part of the plan."""
)

replanner = replanner_prompt | model.with_structured_output(Act)

## Step 5: Create the Graph and Define Nodes
This step creates the workflow graph and defines the nodes (`plan_node`, `agent_node`, `replan_node`). The graph orchestrates the execution of tasks, re-planning, and termination.

In [ ]:
# Create the Graph
async def agent_node(state: PlanExecute) -> PlanExecute:
    """
    Executes the first step in the plan using the agent.

    Args:
        state (PlanExecute): The current state of the workflow.

    Returns:
        PlanExecute: The updated state with the result of the executed step.
    """
    plan = state["plan"]
    plan_str = "\n".join(f"{i+1}. {step}" for i, step in enumerate(plan))
    task = plan[0]
    task_formatted = f"For the following plan:\n{plan_str}\n\nYou are tasked with executing step {1}, {task}."
    agent_response = await agent_executor.ainvoke({"messages": [("user", task_formatted)]})
    return {"past_steps": [(task, agent_response["messages"][-1].content)]}

async def plan_node(state: PlanExecute) -> PlanExecute:
    """
    Generates a plan based on the user's input.

    Args:
        state (PlanExecute): The current state of the workflow.

    Returns:
        PlanExecute: The updated state with the generated plan.
    """
    plan = await planner.ainvoke({"messages": [("user", state["input"])]})
    return {"plan": plan.steps}

async def replan_node(state: PlanExecute) -> PlanExecute:
    """
    Updates the plan based on the results of previous steps.

    Args:
        state (PlanExecute): The current state of the workflow.

    Returns:
        PlanExecute: The updated state with the new plan or final response.
    """
    output = await replanner.ainvoke(state)
    if isinstance(output.action, Response):
        return {"response": output.action.response}
    else:
        return {"plan": output.action.steps}

def should_end(state: PlanExecute) -> Literal["agent_node", END]:
    """
    Determines whether the workflow should end or continue.

    Args:
        state (PlanExecute): The current state of the workflow.

    Returns:
        Literal["agent_node", END]: Returns `END` if the workflow should terminate, otherwise returns `"agent_node"`.
    """
    if "response" in state and state["response"]:
        return END
    else:
        return "agent_node"

workflow = StateGraph(PlanExecute)
workflow.add_node("plan_node", plan_node)
workflow.add_node("agent_node", agent_node)
workflow.add_node("replan_node", replan_node)

workflow.add_edge(START, "plan_node")
workflow.add_edge("plan_node", "agent_node")
workflow.add_edge("agent_node", "replan_node")
workflow.add_conditional_edges("replan_node", should_end, ["agent_node", END])

# Finally, we compile it!
# This compiles it into a LangChain Runnable,
# meaning you can use it as you would any other runnable
app = workflow.compile()

from IPython.display import Image, display
display(Image(app.get_graph(xray=True).draw_mermaid_png()))

## Step 6: Execute the Workflow
This step executes the workflow with a sample input. The agent processes the input, generates a plan, executes tasks, and returns the final result.

In [ ]:
# Example 1: Discovering the Capital City of the 2022 FIFA World Cup Champion
config = {"recursion_limit": 30}
inputs = {"input": "What is the capital city of the country that won the most recent FIFA World Cup?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

In [ ]:
# Example 2: Population of the Largest City in the Country with the Most Olympic Gold Medals
config = {"recursion_limit": 30}
inputs = {"input": "What is the population of the largest city in the country that has won the most Olympic gold medals?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

In [ ]:
# Example 3: Currency of the Country with the Highest GDP in 2023
config = {"recursion_limit": 30}
inputs = {"input": "What is the currency of the country with the highest GDP in 2023?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

In [ ]:
# Example 4: Founder of the Company That Created the Most Popular Smartphone Operating System
config = {"recursion_limit": 30}
inputs = {"input": "Who is the founder of the company that created the most popular smartphone operating system?"}
async for event in app.astream(inputs, config=config):
    for k, v in event.items():
        if k != "__end__":
            print(v)

---

## Conclusion

The example demonstrates how to build a **dynamic, multi-step task execution workflow** using LangChain and LangGraph. By breaking down complex queries into smaller steps, the agent can efficiently handle tasks that require searching for information, processing results, and providing a final response. The workflow is highly flexible, allowing for dynamic re-planning based on the results of previous steps, and can be adapted to a wide range of use cases.

Key takeaways from this example include:
- The importance of **state management** in tracking the progress of a multi-step task.
- The ability to **dynamically re-plan** based on intermediate results, ensuring the workflow adapts to new information.
- The use of **tools** like `DuckDuckGoSearchRun` to gather information and enhance the agent's capabilities.
- The **graph-based workflow** design, which provides a clear and modular structure for defining and executing tasks.

This approach is particularly useful for applications such as **question answering**, **data retrieval**, and **automated decision-making**, where tasks often require multiple steps and dynamic adaptation.